# Tracer flow (single-phase, two-component flow)

## Introduction

This tutorial demonstrates how to set up a single-phase, 2-component model using water
and a tracer. The assumptions on the tracer are that it has no effects on the fluid
properties or flow dynamics, hence a pure tracer. The spreading of the tracer component
throughout the domain is simulated using a hyperbolic transport equation, also referred
to as component mass balance for the tracer.

The full model is implemented in `porepy.examples.tracer_flow` and a step-by-step guide
of the implementation is given here.

## Mathematical model

To understand the required steps and classes, let's first recount the mathematical model.

### Single phase flow

This model aspect is covered in full detail in the respective [tutorial](./single_phase_flow.ipynb).
We recall here that the physics of single phase flow introduce one primary variable, pressure $p$,
and one governing equation, the (total) fluid mass balance:

$$
\dfrac{\partial}{\partial t}\left(\Phi \rho\right) - \nabla\cdot\left( \dfrac{\rho}{\mu} \mathbf{K} \nabla p\right) = 0.
$$

### Transport equation

Porepy utilizes the overall fraction formulation for multi-component fluids.
The mass corresponding to a component $i$ is represented by an overall fraction $z_i$.
One component can arbitrarily be chosen as the reference component and its overall fraction $z_r$
is computed by unity of fractions

$$
z_r = 1 - \sum_{i\neq r} z_i,
$$
i.e. it is a dependent variable. For the sake of our model, we set water as the reference component
and the only remaining quantity of interest is the tracer fraction $z$.

The mass in one phase is in general also split between the present components, leading to
the requirement to introduce partial fractions of a component $i$ in some phase $j$.
But since we assume single-phase flow, the fraction of mass of one component in the single phase
is due to conservation of mass equal to its overall fraction.

This allows us to write the tracer transport equation for the unknown $z$ in the following compact form:

$$
\dfrac{\partial}{\partial t}\left(\Phi \rho z\right) - \nabla\cdot\left( \dfrac{\rho z}{\mu} \mathbf{K} \nabla p\right) = 0.
$$

### Model summary

In a simplified setting with only 1 phase and 2 components, the primary variables of interest are

- pressure $p$ and
- independent overall fraction $z$.

The governing equations are given by

- total mass balance (pressure equation) and
- one component mass balance (tracer transport equation).

The extension of this model to the mixed-dimensional setting is a technicality, which can be omitted here.
In short, the mass flux between subdomains (*interface darcy flux*) is seen as the flux of overall mass.
The mass flux of one component is modelled analogously to the balance equation:
The (total) interface darcy flux is scaled down with the fraction $z$.
The choice of $z$ on the interface is handled via Upwind-coupling.

## Building blocks of the PorePy model

We start by fetching the single-phase flow physics and classes required to extend the model to two components.
Note that the `SinglePhaseFlow` class already includes the pressure variable $p$, the (total) mass balance, and boundary and initial conditions for $p$.

In [27]:
import porepy as pp
from porepy.models.fluid_mass_balance import SinglePhaseFlow

# A mixin handling the introduction of required fractional variables for arbitrary mixtures
from porepy.compositional.compositional_mixins import CompositionalVariables

# Mixins handling BC and IC for fractional variables, and a mixin for the
# component mass balance equation for the tracer.
from porepy.models.compositional_flow import (
    BoundaryConditionsFractions,
    ComponentMassBalanceEquations,
    InitialConditionsFractions,
)

We define the two-component fluid now and use some provided values for water.
For single-phase flow, the only method we need to override is the one defining the
fluid components `get_components`.

This method is originally implemented in the `FluidMixin`, which is already part of the `SinglePhaseFlow`.
It will by default create a single, liquid-like phase.

> Note:
>
>   The water is chosen as the first component, which is set (also by default) as the reference component.

In [28]:
from porepy.applications.material_values.fluid_values import water

class TracerFluid:
    """Setting up a 2-component fluid."""

    def get_components(self) -> list[pp.FluidComponent]:
        """Mixed in method defining water as the reference component and a simple
        tracer as the second component."""

        component_1 = pp.FluidComponent(name="water", **water)
        component_2 = pp.FluidComponent(name="tracer")
        return [component_1, component_2]

As a next step, we set up the domain.
`DomainX` creates a 2D unit square as the matrix with two intersecting line fractures in the shape of an X.
The aim is to impose a left-to-right flow and observing how the left side of the X gets slowely
filled up with the tracer and overflows, assuming that the fractures are impermeable.

In [29]:
import numpy as np
from porepy.applications.md_grids.domains import nd_cube_domain


class DomainX(pp.PorePyModel):

    def set_domain(self) -> None:
        """Setting a 2d unit square as matrix."""
        size = self.units.convert_units(1, "m")
        self._domain = nd_cube_domain(2, size)

    def set_fractures(self) -> None:
        """Setting 2 fractures in x shape."""
        frac_1_points = self.units.convert_units(
            np.array([[0.2, 0.8], [0.2, 0.8]]), "m"
        )
        frac_1 = pp.LineFracture(frac_1_points)
        frac_2_points = self.units.convert_units(
            np.array([[0.2, 0.8], [0.8, 0.2]]), "m"
        )
        frac_2 = pp.LineFracture(frac_2_points)
        self._fractures = [frac_1, frac_2]

    def grid_type(self) -> str:
        return self.params.get("grid_type", "simplex")

    def meshing_arguments(self) -> dict:
        cell_size = self.units.convert_units(0.05, "m")
        cell_size_fracture = self.units.convert_units(0.05, "m")
        mesh_args: dict[str, float] = {
            "cell_size": cell_size,
            "cell_size_fracture": cell_size_fracture,
        }
        return mesh_args


Let's now define initial and boundary conditions.

The following classes do two things:

1. They mix in methods to set IC and BC for pressure, without inheriting the respective
   classes (`InitialConditionsSinglePhaseFlow` and `BoundaryConditionsSinglePhaseFlow`).
   Those classes are already present in the base of `SinglePhaseFlow` and we want the
   methods `initial_pressure` and `bc_values_pressure` to be *pure* mixins. We do not
   want a mess in our hierarchy and a simple MRO.
2. They inherit the IC and BC base classes for fractional variables, and override the
   methods providing trivial values for $z$.

We define inlet and outlet faces for the matrix as a central snippet on the west and east
side respectively.

- The initial pressure is equal to the outlet pressure on the east.
- The inlet pressure is higher than the outlet pressure to enforce a left-to-right flow.
- The initial tracer fraction is zero throughout the domain.
- On the inlet, a tracer concentration of 10% enters the domain.

In [30]:
def inlet_faces(bg: pp.BoundaryGrid, sides: pp.domain.DomainSides) -> np.ndarray:
    """Helper function to define a snippet of the western boundary as the inlet."""

    inlet = np.zeros(bg.num_cells, dtype=bool)
    inlet[sides.west] = True
    inlet &= bg.cell_centers[1] >= 0.4
    inlet &= bg.cell_centers[1] <= 0.6

    return inlet

def outlet_faces(bg: pp.BoundaryGrid, sides: pp.domain.DomainSides) -> np.ndarray:
    """Helper function to define a snippet of the eastern boundary as the inlet."""

    outlet = np.zeros(bg.num_cells, dtype=bool)
    outlet[sides.east] = True
    outlet &= bg.cell_centers[1] >= 0.4
    outlet &= bg.cell_centers[1] <= 0.6

    return outlet


# defining inlet and outlet pressure
p_OUTLET = 1.
p_INLET = 1.5
# defining the amount of tracer comming into the domain
z_inlet = 0.1


class TracerIC(InitialConditionsFractions):

    def initial_pressure(self, sd: pp.Grid) -> np.ndarray:
        """Setting initial pressure equal to pressure on outflow boundary."""
        return np.ones(sd.num_cells) * p_OUTLET
    
    def initial_overall_fraction(self, component: pp.Component, sd: pp.Grid) -> np.ndarray:
        if component.name == 'tracer':
            return np.zeros(sd.num_cells)
        else:
            assert False, "This will never happen since water is a dependent component."


# TODO remove this once fixed
from porepy.models.compositional_flow import _BoundaryConditionsAdvection


class TracerBC(BoundaryConditionsFractions, _BoundaryConditionsAdvection):

    def bc_type_darcy_flux(self, sd: pp.Grid) -> pp.BoundaryCondition:
        """Flagging the inlet and outlet faces as Dirichlet boundary, where pressure
        is given."""
        # Define boundary faces.
        sides = self.domain_boundary_sides(sd)
        bg = self.mdg.subdomain_to_boundary_grid(sd)
        bg_sides = self.domain_boundary_sides(bg)
        inlet = inlet_faces(bg, bg_sides)
        outlet = outlet_faces(bg, bg_sides)

        dirichlet = np.zeros(bg.num_cells, dtype=bool)
        dirichlet[inlet | outlet] = True

        # broadcast to proper size
        dirichlet_faces = np.zeros(sd.num_faces, dtype=bool)
        dirichlet_faces[sides.all_bf] = dirichlet

        return pp.BoundaryCondition(sd, dirichlet_faces, "dir")

    def bc_values_pressure(self, bg: pp.BoundaryGrid) -> np.ndarray:
        """Defines some non-trivial values on inlet and outlet faces of the matrix."""

        p = np.zeros(bg.num_cells)

        # defining BC values only on matrix
        if bg.parent.dim == 2:
            sides = self.domain_boundary_sides(bg)
            inlet = inlet_faces(bg, sides)
            outlet = outlet_faces(bg, sides)

            p[inlet] = p_INLET
            p[outlet] = p_OUTLET

        return p
    
    def bc_values_overall_fraction(self, component: pp.Component, bg: pp.BoundaryGrid) -> np.ndarray:
        """Defines some non-trivial inflow of the tracer component on the inlet."""

        z = np.zeros(bg.num_cells)

        if bg.parent.dim == 2:
            if component.name == 'tracer':
                sides = self.domain_boundary_sides(bg)
                inlet = inlet_faces(bg, sides)
                
                z[inlet] = z_inlet
            else:
                assert False, "This will never happen since water is a dependent component."

        return z


Finally, let's define the model set-up using above building blocks.

- The domain and the fluid are pure mixins, in the sense that they override some methods
  which are already part of `SinglePhaseFlow`.
- The `CompositionalVariables` mixin includes functionality to create the tracer
fraction $z$.
- The `ComponentMassBalanceEquations` will add transport equations for each independent
component, which is the tracer in our case.
- `TracerIC` and `TracerBC` provide IC and BC for $p$ and $z$. Using the functionality
of `super()` in various mixed in IC and BC classes, the set-up will first call the IC/BC
for fractions, and then for pressure. This is due to `SinglePhaseFlow` being put at the
bottom of the sequence of base classes.
- Lastly, `SinglePhaseFlow` also contains constitutive laws for the phase density and
viscosity. We exploit the default implementation which uses heuristic functions and
accesses the `compressibility` and `viscosity` of the water component. I.e., the tracer
has no effect on the thermodynamic properties of the fluid.

In [31]:

class TracerFlowSetup(
    DomainX,
    TracerFluid,
    CompositionalVariables,
    ComponentMassBalanceEquations,
    TracerBC,
    TracerIC,
    SinglePhaseFlow,
):
    """Complete set-up for tracer flow modelled as a single phase, 2-component flow
    problem."""

## Run simulation

The model is created and we are ready for the simulation.

In [32]:
solid = pp.SolidConstants(porosity=0.1, permeability=1e-11, normal_permeability=1e-19)

params = {
    'material_constants' : {
        'solid': solid
    },
}
days = 365
t_scale = 1e-5
T_end = 40 * days * t_scale
dt_init = 1 * days * t_scale
max_iterations = 80
newton_tol = 1e-6
newton_tol_increment = newton_tol

time_manager = pp.TimeManager(
    schedule=[0, T_end],
    dt_init=dt_init,
    dt_min_max=(0.1 * dt_init, 2 * dt_init),
    iter_max=max_iterations,
    iter_optimal_range=(2, 10),
    iter_relax_factors=(0.9, 1.1),
    recomp_factor=0.1,
    recomp_max=5,
    print_info=True,
)

## What we have explored

1. We have shown how to create a 2-component fluid.
2. We have explored the relevant building blocks for a single-phase, 2-component isothermal flow model with heuristic thermodynamic properties.
3. The extension to multi-component fluids is trivial. Just add more components and respective IC & BC.
4. we have shown the parallels between the mathematical model and representations of individual aspects as PorePy classes. 